In [1]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install biopython

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [46]:
from Bio import Align
from Bio.Align import substitution_matrices

In [39]:
import numpy as np

In [57]:
seq1 = "ATCGTAGC"
seq2 = "ATCGTACG"
seq3 = "GCTAGCTA"
seq4 = "TACGATCG"
rna1 = "AUGCUAGC"
rna2 = "AUGCUGCA"
rna3 = "GCUAGCUA"
rna4 = "UACGAUCG"
protein1 = "ACDEFGHIKLMNPQRSTVWY"
protein2 = "ACDFGHIKLMNPRSTVWY"
protein3 = "ACDEFGHLMNPQRSTVWX"
protein4 = "ABCDEFGHIJKLMN"

In [55]:
def needleman_wunsch(seq_1, seq_2, match = 1, mismatch = -1, gap = -2):
    m = len(seq_1)
    n = len(seq_2)
    score = np.zeros(shape = (n, m))
    arr = np.zeros(shape = (n + 1, m + 1))
    arr[0] += np.arange(0, m + 1)*gap
    arr[:, 0] += np.arange(0, n + 1)*gap
    
    for i in range(n):
        for j in range(m):
            score[i][j] =  match if seq_1[j] == seq_2[i] else mismatch
        
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            arr[i][j] = max(arr[i-1][j] + gap, arr[i][j-1] + gap, arr[i-1][j-1] + score[i-1][j-1])
    
    seq_1_al = ''
    seq_2_al = ''
    
    i, j = len(seq_2), len(seq_1)
    while not(i == 0 and j == 0): 
        if i > 0 and j > 0 and arr[i][j] == arr[i-1][j-1] + score[i-1][j-1]:                  
            seq_1_al = seq_1_al + seq_1[j-1]
            seq_2_al = seq_2_al + seq_2[i-1]
            i -= 1
            j -= 1
        elif i >= 0 and arr[i][j] == arr[i-1][j] + gap:                  
            seq_1_al = seq_1_al + '-'
            seq_2_al = seq_2_al + seq_2[i-1]
            i -= 1
        elif j >= 0 and arr[i][j] == arr[i][j-1] + gap:        
            seq_1_al = seq_1_al + seq_1[j-1]
            seq_2_al = seq_2_al + '-'
            j -= 1
    
    print(seq_1_al[::-1])
    print(seq_2_al[::-1])

rna1 = "AUGCUAGC"
rna2 = "AUGCUGCA"
rna3 = "GCUAGCUA"
rna4 = "UACGAUCG"

needleman_wunsch(rna1, rna2)  
print()
needleman_wunsch(rna1, rna3)   
print()
needleman_wunsch(rna3, rna4) 
print()
needleman_wunsch("GACGAAG", "ACCAAG")

AUGCUAGC-
AUGCU-GCA

AUGCUAGC--
--GCUAGCUA

-GCUAGCUA
UACGAUC-G

GACGAAG
-ACCAAG


In [61]:
def smith_waterman(seq_1, seq_2, match = 1, mismatch = -1, gap = -2):
    m = len(seq_1)
    n = len(seq_2)
    score = np.zeros(shape = (n, m))
    arr = np.zeros(shape = (n + 1, m + 1))
    
    for i in range(n):
        for j in range(m):
            score[i][j] =  match if seq_1[j] == seq_2[i] else mismatch
        
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            arr[i][j] = max(arr[i-1][j] + gap, arr[i][j-1] + gap, arr[i-1][j-1] + score[i-1][j-1], 0)
    
    seq_1_al = ''
    seq_2_al = ''
    
    i, j = np.unravel_index(arr.argmax(), arr.shape)
    while not((i == 0 and j == 0) or arr[i][j] == 0): 
        if i > 0 and j > 0 and arr[i][j] == arr[i-1][j-1] + score[i-1][j-1]:                  
            seq_1_al = seq_1_al + seq_1[j-1]
            seq_2_al = seq_2_al + seq_2[i-1]
            i -= 1
            j -= 1
        elif i >= 0 and arr[i][j] == arr[i-1][j] + gap:                  
            seq_1_al = seq_1_al + '-'
            seq_2_al = seq_2_al + seq_2[i-1]
            i -= 1
        elif j >= 0 and arr[i][j] == arr[i][j-1] + gap:        
            seq_1_al = seq_1_al + seq_1[j-1]
            seq_2_al = seq_2_al + '-'
            j -= 1
    
    print(seq_1_al[::-1])
    print(seq_2_al[::-1])

rna1 = "AUGCUAGC"
rna2 = "AUGCUGCA"
rna3 = "GCUAGCUA"
rna4 = "UACGAUCG"

smith_waterman(rna1, rna2)  
print()
smith_waterman(rna1, rna3)  
print()
smith_waterman("GACGAAG", "ACCAAG")
print()
smith_waterman("AUGTUCGCAUUAGCAGGCCUAAUGGCACCGUCCAGCUCGUAUGCAGUGCA", "AGGCCGAAUGGUCAC")

AUGCU
AUGCU

GCUAGC
GCUAGC

ACGAAG
ACCAAG

AGGCCUAAUGG-CAC
AGGCCGAAUGGUCAC


In [54]:
# check with biopython Align
# Мне этот инструмент не очень нравится, он не очень хорошо делает локальное выравнивание, если не установить огромный штраф за открытие гэпа
# Но для небольших похожих последовательностей подходит
seq_1 = "AUGTUCGCAUUAGCAGGCCUAAUGGCACCGUCCAGCUCGUAUGCAGUGCA"
seq_2 = "GUCCUGCUCG"
seq_3 = "GACGAAG"
seq_4 = "ACCAAG"

aligner_local = Align.PairwiseAligner(match_score = 2.0, mismatch_score = -1)
aligner_local.open_gap_score = -10
aligner_local.extend_gap_score = -1
aligner_local.mode = 'local'

aligner_global = Align.PairwiseAligner(match_score = 2.0, mismatch_score = -1)
aligner.open_gap_score = -3
aligner.extend_gap_score = -1
aligner.mode = 'global'

alignments_1 = aligner_global.align(seq_1, seq_2)
alignments_2 = aligner_local.align(seq_1, seq_2)
alignments_3 = aligner_global.align(seq_3, seq_4)

print(alignments_1[0])   # здесь огромное количество разных выравниваний, поэтому вывожу только первое; видим, что оно глобальное
print(alignments_1[0].score)

print(alignments_2[0])   # здесь выравнивание всего одно и оно локальное, очень компактное
print(alignments_2[0].score)

for align in alignments_3:  # здесь всего три варианта; как и было сказано, на коротких последовательностях работает нормально
    print(align)
    print(align.score)

target            0 AUGTUCGCAUUAGCAGGCCUAAUGGCACCGUCCAGCUCGUAUGCAGUGCA 50
                  0 --|-||-|-|--||-----|-----|---|-------------------- 50
query             0 --G-UC-C-U--GC-----U-----C---G-------------------- 10

20.0
target           29 GUCCAGCUCG 39
                  0 ||||.||||| 10
query             0 GUCCUGCUCG 10

17.0
target            0 GACG-AAG 7
                  0 -||--||| 8
query             0 -AC-CAAG 6

10.0
target            0 GAC-GAAG 7
                  0 -||--||| 8
query             0 -ACC-AAG 6

10.0
target            0 GA-CGAAG 7
                  0 -|-|-||| 8
query             0 -ACC-AAG 6

10.0


In [48]:
#этот же инструмент работает и для белков, если задать матрицу замен BLOSUM 
aligner = Align.PairwiseAligner()
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
aligner.open_gap_score = -10
aligner.extend_gap_score = -0.5

alignment = aligner.align(protein1, protein2)
print(alignment[0])
print(alignment[0].score)

target            0 ACDEFGHIKLMNPQRSTVWY 20
                  0 |||-|||||||||-|||||| 20
query             0 ACD-FGHIKLMNP-RSTVWY 18

86.0


Видим, что ручное выравнивание и выравнивание алгоритмом Нидлмана-Вунша совпали. Алгоритм Смита-Ватермана производит локальное выравнивание, поэтому выдает в основном общие куски. Реализованный пакет Biopython выдает несколько результатов, и все они более-менее совпадают с полученными другими методами. 